In [26]:
import numpy as np
import pandas as pd
from WordleFunctions import wordle, cypher, entropy

five = pd.concat([
    pd.read_csv("wordle-answers-alphabetical.txt",header=None,names=["word"]),
    pd.read_csv("wordle-allowed-guesses.txt",header=None,names=["word"])],axis=0,ignore_index=True
)
answers = pd.read_csv("wordle-answers-alphabetical.txt",header=None,names=["word"])

def nlogn(n):
    return (1-1/n)*np.log(1-1/n)

def nextQuordle(df1,df2,df3,df4,allWords):
    ans=""
    m=0
    for i in range(len(allWords)):
        word=allWords.iloc[i] # next guess under consideration
        e=entropy(cypher(df1,word))+entropy(cypher(df2,word))+entropy(cypher(df3,word))+entropy(cypher(df4,word))
        if i in df1.index:
            e-=nlogn(len(df1))
        if i in df2.index:
            e-=nlogn(len(df2))
        if i in df3.index:
            e-=nlogn(len(df3))
        if i in df4.index:
            e-=nlogn(len(df4))
        if e>m:
            m=e
            ans=word
    return ans

# prompt user to respond with the Wordle response, e.g. as "BYYBG"
def getResponse(prompt):
    while(True):
        print(prompt)
        string=input()
        string=string.upper()
        if len(string)==0:
            return ""
        if len(string)!=5:
            print("Response must be exactly 5 characters")
        else:
            done=1
            for char in string:
                if char not in ("B","G","Y"):
                    print("Enter Wordle response using 'B','Y','G'")
                    done=0
                    break
            if done==1:
                break
    return string

# Recursively find the best word until solved
def quordleSolver(dfPossibleAnswers):
    ans1=dfPossibleAnswers.copy()
    ans2=dfPossibleAnswers.copy()
    ans3=dfPossibleAnswers.copy()
    ans4=dfPossibleAnswers.copy()
    guess="soare"
    fourSignals=["" for i in range(4)]
    solved=[0 for i in range(4)]
    done=0
    while(True):
        print("Next guess:")
        print("\t",guess)
        for i in range(4):
            if solved[i]==0:
                fourSignals[i]=getResponse("Enter response " + str(i+1) +":")
                if fourSignals[i]=="":
                    done=1
                    break
                if fourSignals[i]=="GGGGG":
                    solved[i]=1
                    if sum(solved)==4:
                        done=1
                        break
        if done==1:
            break
        ans1=ans1[cypher(ans1.word,guess) == fourSignals[0]]
        ans2=ans2[cypher(ans2.word,guess) == fourSignals[1]]
        ans3=ans3[cypher(ans3.word,guess) == fourSignals[2]]
        ans4=ans4[cypher(ans4.word,guess) == fourSignals[3]]
        if ans1.shape[0]==1 and ans2.shape[0]==1 and ans3.shape[0]==1 and ans4.shape[0]==0:
            print("Then the words must be:")
            print("\t",ans1.word.iloc[0],ans2.word.iloc[0],ans3.word.iloc[0],ans4.word.iloc[0])
            break
        if (ans1.shape[0]==0 and solved[0]==0) or (ans2.shape[0]==0 and solved[1]==0) \
            or (ans3.shape[0]==0 and solved[2]==0) or (ans4.shape[0]==0 and solved[3]==0):
            print("No such word was found.")
            break
        if ans1.shape[0]==1 and solved[0]==0:
            print("Word 1 found!")
            guess=ans1.word.iloc[0]
            solved[0]=1
        elif ans2.shape[0]==1 and solved[1]==0:
            print("Word 2 found!")
            guess=ans2.word.iloc[0]
            solved[1]=1
        elif ans3.shape[0]==1 and solved[2]==0:
            print("Word 3 found!")
            guess=ans3.word.iloc[0]
            solved[2]=1
        elif ans4.shape[0]==1 and solved[3]==0:
            print("Word 4 found!")
            guess=ans4.word.iloc[0]
            solved[3]=1
        else:
            guess=nextQuordle(ans1.word,ans2.word,ans3.word,ans4.word,five.word)
        if sum(solved)==4:
            print("Then the words must be:")
            print("\t",ans1.word.iloc[0],ans2.word.iloc[0],ans3.word.iloc[0],ans4.word.iloc[0])
            break

In [27]:
quordleSolver(five)

Next guess:
	 soare
Enter response 1:
bbgyg
Enter response 2:
yybbg
Enter response 3:
bbgbb
Enter response 4:
bbbyg
Next guess:
	 gitch
Enter response 1:
bbbbb
Enter response 2:
bbbyb
Enter response 3:
bybbb
Enter response 4:
bbbbb
Word 2 found!
Next guess:
	 close
Enter response 1:
bbbbg
Enter response 3:
bgbbb
Enter response 4:
bbbbg
Next guess:
	 pendu
Enter response 1:
bybbb
Enter response 3:
bbbbb
Enter response 4:
gybgy
Word 3 found!
Next guess:
	 flail
Enter response 1:
bbgbb
Enter response 4:
bbbbb
Word 4 found!
Next guess:
	 prude
Enter response 1:
bgbbg
Next guess:
	 amaze
Enter response 1:
bbgbg
Next guess:
	 brake
Enter response 1:
gggbg
Word 1 found!
